In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler

In [3]:
# Veriyi yükle
df = pd.read_csv('../data/Spotify_dataset.csv')

In [4]:
# Basit temizlik: Track ID'si aynı olanları sil (Tekrar öneri yapmamak için)
# Baseline aşamasında karmaşık temizlik yapmıyoruz, sadece çalışmasını sağlıyoruz.
df = df.drop_duplicates(subset=['track_id']).reset_index(drop=True)

print(f"Baseline için kullanılacak veri boyutu: {df.shape}")

Baseline için kullanılacak veri boyutu: (89741, 21)


In [5]:
# Sadece sayısal ses özelliklerini seçiyoruz
feature_cols = [
    'danceability', 'energy', 'loudness', 'speechiness', 
    'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo'
]

# Eksik kolon var mı kontrol et
available_features = [col for col in feature_cols if col in df.columns]
print(f"Kullanılan Özellikler: {available_features}")

# Feature matrisini oluştur
X = df[available_features]

# NaN değer varsa baseline hatasız çalışsın diye ortalama ile doldur (Simple Imputation)
X = X.fillna(X.mean())

Kullanılan Özellikler: ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']


In [6]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print("Veri ölçeklendi. Örnek satır (ilk 5 özellik):\n", X_scaled[0][:5])

Veri ölçeklendi. Örnek satır (ilk 5 özellik):
 [ 0.64425952 -0.67597615  0.33573068  0.49046351 -0.87517711]


In [7]:
# Cosine Similarity (Kosinüs Benzerliği) kullanıyoruz.
# Müzik önerilerinde 'Euclidean' yerine 'Cosine' genelde daha iyi sonuç verir çünkü vektörlerin yönüne (içeriğine) bakar.
model = NearestNeighbors(n_neighbors=10, algorithm='brute', metric='cosine')
model.fit(X_scaled)

print("Baseline Model Eğitildi.")

Baseline Model Eğitildi.


In [10]:
def recommend_song(song_name, artist_name=None):
    # 1. Şarkıyı veri setinde bul
    if artist_name:
        # Hem şarkı hem sanatçı eşleşmesi ara (büyük/küçük harf duyarsız)
        mask = (df['track_name'].str.lower() == song_name.lower()) & \
               (df['artists'].str.lower().str.contains(artist_name.lower(), na=False))
    else:
        # Sadece şarkı ismi ara
        mask = df['track_name'].str.lower() == song_name.lower()
    
    if mask.sum() == 0:
        print(f"Hata: '{song_name}' veri setinde bulunamadı.")
        return None
    
    # İlk eşleşmeyi al
    song_idx = df[mask].index[0]
    song_data = df.iloc[song_idx]
    
    print(f"Seçilen Şarkı: {song_data['track_name']} - {song_data['artists']}")
    
    # 2. Modelden benzerlerini bul
    # reshape(1, -1) tek bir örnek olduğunu belirtir
    distances, indices = model.kneighbors(X_scaled[song_idx].reshape(1, -1))
    
    # 3. Sonuçları listele
    print("\n--- ÖNERİLEN ŞARKILAR (BASELINE) ---")
    # indices[0][1:] -> 0. indeks şarkının kendisidir, onu atlıyoruz
    recommendations = df.iloc[indices[0][1:]]
    
    for i, (idx, row) in enumerate(recommendations.iterrows(), 1):
        dist = distances[0][i]
        print(f"{i}. {row['track_name']} - {row['artists']} (Mesafe: {dist:.4f})")
        
    return recommendations

In [11]:
# Örnek Test: Adele veya veri setinde var olduğundan emin olduğun bir şarkı
# Not: Eğer hata alırsan veri setindeki rastgele bir ismi kullanmak için: df['track_name'].sample(1) yapabilirsin.
test_song = "Blinding Lights" # Veya df['track_name'].iloc[100]
recommend_song(test_song)

Seçilen Şarkı: Blinding Lights - Kidz Bop Kids

--- ÖNERİLEN ŞARKILAR (BASELINE) ---
1. Atrakce - PAWLIE POIZN;Medooza (Mesafe: 0.0103)
2. Me Voy - Los Victorios (Mesafe: 0.0107)
3. Mob Rule - Bad//Dreems (Mesafe: 0.0121)
4. Benmişim - Nev (Mesafe: 0.0123)
5. My Person - Spencer Crandall (Mesafe: 0.0124)
6. You Are the Best Thing - Ray LaMontagne (Mesafe: 0.0128)
7. My Person - Spencer Crandall (Mesafe: 0.0131)
8. PLAYER - PASSEPIED (Mesafe: 0.0135)
9. Vaanampaadiyin - Sujatha (Mesafe: 0.0143)


,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
30348,33591,6wtELTbJEAkbugVDSKa0sh,PAWLIE POIZN;Medooza,Atrakce,Atrakce,36,181073,True,0.622,0.735,...,-5.664,1,0.0470,0.06690,0.000000,0.1100,0.947,169.912,4,emo
79556,100553,0SFETrpna5wpB1ibBPPyWI,Los Victorios,2,Me Voy,34,204875,False,0.638,0.869,...,-5.137,1,0.0600,0.01410,0.000000,0.1100,0.953,172.001,4,ska
34699,38541,1sQie71FoBxfYusuTDOXAA,Bad//Dreems,Gutful,Mob Rule,29,161500,False,0.581,0.794,...,-4.384,1,0.0426,0.03290,0.000021,0.1660,0.892,166.936,4,garage
87999,112065,74sGontnRy751y9wYos5Fi,Nev,Sen Gibi,Benmişim,49,264939,False,0.566,0.840,...,-4.548,1,0.0343,0.00625,0.000000,0.0699,0.900,174.041,4,turkish
18776,19863,1MOOJuxUu9QiQE9GgkYYPb,Spencer Crandall,Wilderness,My Person,61,177000,False,0.567,0.753,...,-5.291,1,0.0487,0.08940,0.000000,0.1300,0.756,165.966,4,country
444,444,1jyddn36UN4tVsJGtaJfem,Ray LaMontagne,Gossip In The Grain,You Are the Best Thing,67,231840,False,0.578,0.727,...,-4.872,1,0.0332,0.03040,0.000020,0.1720,0.885,170.593,4,acoustic
18539,19603,1K61P0kbiT6cJzh77NnxFg,Spencer Crandall,My Person,My Person,55,177000,False,0.567,0.757,...,-5.290,1,0.0485,0.09360,0.000000,0.1300,0.754,166.010,4,country
52836,61565,1rW8jqQFuxtUOfpRZaIZQR,PASSEPIED,ニュイ,PLAYER,21,263474,False,0.617,0.839,...,-5.192,1,0.0366,0.02400,0.034200,0.0675,0.846,170.020,4,j-idol
58775,69744,4jpg2Qise04xi5qoxsfre2,Sujatha,Unnidathil Ennai Koduthen (Original Motion Pic...,Vaanampaadiyin,22,255880,False,0.589,0.826,...,-7.220,0,0.0420,0.08360,0.000017,0.1420,0.860,169.768,4,malay


## Baseline Model Raporu

### 1. Kullanılan Yaklaşım
- **Algoritma:** Unsupervised Nearest Neighbors (KNN).
- **Metrik:** Cosine Similarity (Kosinüs Benzerliği).
- **Feature Set:** Sadece ham ses özellikleri (danceability, energy, loudness, tempo, vb.). Tür (Genre) veya Sanatçı bilgisi kullanılmadı.

### 2. Gözlemler
- Model, ritmik özellikleri (tempo, energy) benzer olan şarkıları bulmakta başarılı.
- **Eksiklik:** Ancak sadece ses özelliklerine baktığı için, "Rock" dinleyen birine benzer tempoda "Pop" şarkısı önerebiliyor. Tür uyumu (Genre consistency) şu an düşük.
- **Hedef:** Bir sonraki aşamada (Feature Engineering), tür bilgisini de modele katarak (One-Hot Encoding veya Frequency Encoding) bu sorunu çözeceğiz.